In [1]:
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pdb import set_trace as db

In [2]:
class Stock:
    def __init__(self,name,start,end,comparator=None):
        self.name = name
        print('getting ticker for ' + name + '...')
        self.ticker = yf.Ticker(name)
        self.df = self.ticker.history(period='1d', start=start, end=end)
        print('Succesfully got ticker for ' + name)
        #db()
        self.vals = self.df['Close'].values
        self.time = self.df.index
        if comparator:
            self.valsCompared = [v/n for v,n in zip(self.vals,comparator.valsNorm)]
        else:
            self.valsCompared = self.vals
        self.valsNorm = self.valsCompared/self.valsCompared[0]
        
class Stocks:
    def __init__(
            self,
            *args,
            start = datetime.now() - relativedelta(years=1),
            end = datetime.now(),
            comparatorName = None
        ):
        self.listOfStocks = []
        if comparatorName:
            self.comparator = Stock(comparatorName,start=start,end=end)
        else:
            self.comparator = None
        for arg in args:
            self.listOfStocks.append(Stock(arg,start=start,end=end,comparator=self.comparator))

In [3]:
# Set start and end as one year ago to now
endDate = datetime.now()
startDate = endDate - relativedelta(years=1)
# Create stocks object
stocks = Stocks('MSFT','AAPL','GOOGL',start=startDate,end=endDate)

getting ticker for MSFT...
Succesfully got ticker for MSFT
getting ticker for AAPL...
Succesfully got ticker for AAPL
getting ticker for GOOGL...
Succesfully got ticker for GOOGL


In [4]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected = True)

data = []
for stock in stocks.listOfStocks:
    data.append(
        go.Scatter(
            x = stock.time,
            y = stock.valsNorm,
            name = stock.name
        )
    )
layout = go.Layout(
  title = 'Normalized Stock Performance',
  yaxis = dict(
    hoverformat = '.3g'
  )
)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

In [5]:
# Same stocks but this time baselined against the S&P 500 (comparator:VTI)
stocks = Stocks('MSFT','AAPL','GOOGL',start=startDate,end=endDate,comparatorName='VTI')

getting ticker for VTI...
Succesfully got ticker for VTI
getting ticker for MSFT...
Succesfully got ticker for MSFT
getting ticker for AAPL...
Succesfully got ticker for AAPL
getting ticker for GOOGL...
Succesfully got ticker for GOOGL


In [6]:
data = []
for stock in stocks.listOfStocks:
    data.append(
        go.Scatter(
            x = stock.time,
            y = stock.valsNorm,
            name = stock.name
        )
    )
layout = go.Layout(
  title = 'Normalized Stock Performance Relative to S&P 500',
  yaxis = dict(
    hoverformat = '.3g'
  )
)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)